## Imports, Load model & data

In [4]:
# list of libraries to import
import os
import urllib.request
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import cv2
import scipy.io
%matplotlib inline

import os
from sklearn.utils import shuffle
from sklearn.cross_validation import train_test_split

from keras import backend as K
K.set_image_dim_ordering('tf')

from keras.utils import np_utils
from keras.models import Sequential
from keras.layers.core import Dense, Dropout, Activation, Flatten
from keras.layers.convolutional import Convolution2D, MaxPooling2D
from keras.optimizers import SGD, RMSprop, adam

/home/anto/anaconda3/lib/python3.6/site-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)
Using TensorFlow backend.


In [121]:
# load model & output column Desc
from keras.models import load_model
model=load_model('model_v3.hdf5')

#Y_cols = scipy.io.loadmat('Y_columns.mat')['out']
#int(float(Y_cols[0,3][0].split('_')[1]))

In [22]:
# load data
X_lbl = 'test_data/img_data_'+str(0)+'.mat'
img_id_lbl = 'test_data/img_id_'+str(0)+'.mat'
X = scipy.io.loadmat(X_lbl)["out"].astype('float32')
img_id = scipy.io.loadmat(img_id_lbl)["out"].astype('float32')

for i in range(1,13):
    print(i)
    X_lbl = 'test_data/img_data_'+str(i)+'.mat'
    img_id_lbl = 'test_data/img_id_'+str(i)+'.mat'
    interim_X = scipy.io.loadmat(X_lbl)["out"].astype('float32')
    interim_Y = scipy.io.loadmat(img_id_lbl)["out"].astype('float32')
    X = np.concatenate((X,interim_X),axis=0)
    img_id = np.concatenate((img_id,interim_Y),axis=1)
    print(i)    
    interim_X = []
    interim_Y = []
    
# load output column meaning
#Y_cols = 

1
1
2
2
3
3
4
4
5
5
6
6
7
7
8
8
9
9
10
10
11
11
12
12


In [27]:
print(X.shape)
img_id.shape
X.max()
X /= 255
X = np.expand_dims(X, axis=4)

(12110, 128, 128)


## Predict and write to Pandas DF

In [122]:
Y_pred = model.predict(X)

In [123]:
Y_pred.shape
(img_id[0,:]).astype('int')

array([    1,     2,     3, ..., 12798, 12799, 12800])

In [124]:
def find_class(df):
    #cols = df.columns
    for idx in range(len(df)):
        if df[idx] == df.max():
            return int(float(Y_cols[0,idx][0].split('_')[1]))

In [125]:
df_predict = pd.DataFrame()
df_predict["id"]=img_id[0,:].astype('int')

In [110]:
df_predict.head(5)

,id
0,1
1,2
2,3
3,4
4,5


In [126]:
Y_pd_predict = pd.DataFrame(Y_pred)
df_predict["predicted"] = Y_pd_predict.apply(find_class, axis=1)

In [127]:
df_predict_sorted = df_predict.sort_values(by=["id"],ascending=True)

In [113]:
df_predict_sorted.head(5)

,id,predicted
0,1,40
1,2,92
2,3,65
3,4,65
4,5,76


In [92]:
df_predict_sorted.iloc[9]["id"]

11

In [128]:
missing =[]
idx=0
for i in range(1,12801):
    if not(df_predict_sorted.loc[idx]["id"] == i):
        #j=i-df_predict_sorted.loc[idx]["id"]
        missing.append(i)
    else:
        idx+=1

In [129]:
len(missing)+df_predict_sorted.shape[0]

12800

In [130]:
df_missing=pd.DataFrame()
df_missing["id"]=missing

In [131]:
df_missing["predicted"]=5

In [118]:
df_missing

,id,predicted
0,10,5
1,42,5
2,58,5
3,60,5
4,80,5
5,95,5
6,129,5
7,152,5
8,156,5
9,166,5


In [132]:
df_all = pd.concat([df_predict_sorted,df_missing])

In [133]:
df_all.to_csv('soln_NN_v4.csv', index=False)